In [3]:
import os
from collections import deque
from dotenv import load_dotenv
from google import genai
import pandas as pd
from pydantic import BaseModel, Field
import time
from typing import Annotated, Literal

folder_path = 'data/europe/'

ieee_file = folder_path + 'ieee_europe.csv'
scopus_file = folder_path + 'scopus_europe.csv'
wos_file = folder_path + 'wos_europe.xls'

# IEEE Xplore

In [4]:
df_ieee = pd.read_csv(ieee_file)
df_ieee

,Document Title,Authors,Author Affiliations,Publication Title,Date Added To Xplore,Publication Year,Volume,Issue,Start Page,End Page,...,Mesh_Terms,Article Citation Count,Patent Citation Count,Reference Count,License,Online Date,Issue Date,Meeting Date,Publisher,Document Identifier
0,Comparative Analysis of Different Classifiers ...,N. Vyas; G. K. Sethi; S. Singh,"Department of Computer Science, Punjabi Univer...",2024 4th International Conference on Technolog...,21 Jan 2025,2024,NaN,NaN,1173,1177,...,NaN,4.0,NaN,19,IEEE,21 Jan 2025,NaN,NaN,IEEE,IEEE Conferences
1,SARSense: Analyzing air- and space-borne C- an...,D. Mengen; C. Montzka; T. Jagdhuber; A. Fluhre...,"Forschungszentrum Jülich, Institute of Bio-and...",2021 IEEE International Geoscience and Remote ...,12 Oct 2021,2021,NaN,NaN,6092,6095,...,NaN,NaN,NaN,9,IEEE,12 Oct 2021,NaN,NaN,IEEE,IEEE Conferences
2,Agricultural SandboxNL: A Crop Parcel Level Da...,V. Kumar; M. Huber; M. Shorachi; B. Rommen; S....,"Department of Water Management, Delft Universi...",2021 IEEE International Geoscience and Remote ...,12 Oct 2021,2021,NaN,NaN,6284,6287,...,NaN,1.0,NaN,14,IEEE,12 Oct 2021,NaN,NaN,IEEE,IEEE Conferences
3,Open weather data evaluation for crop irrigati...,T. Karamitsou; D. Seventekidis; C. Karapiperis...,Agrosense S.A Research and Development of Netw...,"2022 7th South-East Europe Design Automation, ...",1 Nov 2022,2022,NaN,NaN,1,4,...,NaN,3.0,NaN,14,IEEE,1 Nov 2022,NaN,NaN,IEEE,IEEE Conferences
4,A New European Settlement Map From Optical Rem...,A. J. Florczyk; S. Ferri; V. Syrris; T. Kemper...,Institute for the Protection and Security of t...,IEEE Journal of Selected Topics in Applied Ear...,19 May 2017,2016,9,5.0,1978,1992,...,NaN,43.0,NaN,55,EU,28 Oct 2015,NaN,NaN,IEEE,IEEE Journals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Remote Sensing For Assessing Drought Insurance...,K. Heidler; A. Fietzke,"Technische Universität München, Munich, German...",IGARSS 2019 - 2019 IEEE International Geoscien...,14 Nov 2019,2019,NaN,NaN,7306,7309,...,NaN,1.0,NaN,13,IEEE,14 Nov 2019,NaN,NaN,IEEE,IEEE Conferences
117,"Automated Individual Pig Localisation, Trackin...",J. Cowton; I. Kyriazakis; J. Bacardit,"School of Natural and Environmental Sciences, ...",IEEE Access,14 Aug 2019,2019,7,NaN,108049,108060,...,NaN,91.0,NaN,40,CCBY,5 Aug 2019,NaN,NaN,IEEE,IEEE Journals
118,An Approximate Spectral Clustering Ensemble fo...,K. Taşdemir; Y. Moazzen; I. Yildirim,"Department of Computer Engineering, Antalya In...",IEEE Journal of Selected Topics in Applied Ear...,19 May 2017,2015,8,5.0,1996,2004,...,NaN,15.0,NaN,42,IEEE,7 May 2015,NaN,NaN,IEEE,IEEE Journals
119,ICFHR2016 Competition on the Classification of...,F. Cloppet; V. Églin; V. C. Kieu; D. Stutzmann...,"LIPADE, Paris Descartes University, Paris, Fra...",2016 15th International Conference on Frontier...,16 Jan 2017,2016,NaN,NaN,590,595,...,NaN,26.0,NaN,8,IEEE,16 Jan 2017,NaN,NaN,IEEE,IEEE Conferences


In [5]:
df_ieee = df_ieee[['DOI', 'Document Title', 'Authors', 'Publication Year', 'Abstract', 'Article Citation Count']]

df_ieee = df_ieee.rename(columns={
    'Document Title': 'Title',
    'Publication Year': 'Year',
    'Article Citation Count': 'Citations'
})

df_ieee

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/ICTACS62700.2024.10840995,Comparative Analysis of Different Classifiers ...,N. Vyas; G. K. Sethi; S. Singh,2024,Image fusion could adopt the optical and micro...,4.0
1,10.1109/IGARSS47720.2021.9553399,SARSense: Analyzing air- and space-borne C- an...,D. Mengen; C. Montzka; T. Jagdhuber; A. Fluhre...,2021,The upcoming launch of the L-band Synthetic Ap...,NaN
2,10.1109/IGARSS47720.2021.9553571,Agricultural SandboxNL: A Crop Parcel Level Da...,V. Kumar; M. Huber; M. Shorachi; B. Rommen; S....,2021,The systematic high temporal coverage of Senti...,1.0
3,10.1109/SEEDA-CECNSM57760.2022.9932913,Open weather data evaluation for crop irrigati...,T. Karamitsou; D. Seventekidis; C. Karapiperis...,2022,Treated wastewater reuse is increasingly impor...,3.0
4,10.1109/JSTARS.2015.2485662,A New European Settlement Map From Optical Rem...,A. J. Florczyk; S. Ferri; V. Syrris; T. Kemper...,2016,An application of a general methodology for pr...,43.0
...,...,...,...,...,...,...
116,10.1109/IGARSS.2019.8898926,Remote Sensing For Assessing Drought Insurance...,K. Heidler; A. Fietzke,2019,"In this study, the viability of assessing drou...",1.0
117,10.1109/ACCESS.2019.2933060,"Automated Individual Pig Localisation, Trackin...",J. Cowton; I. Kyriazakis; J. Bacardit,2019,Individual pig tracking is key to stepping awa...,91.0
118,10.1109/JSTARS.2015.2424292,An Approximate Spectral Clustering Ensemble fo...,K. Taşdemir; Y. Moazzen; I. Yildirim,2015,Unsupervised clustering of high spatial resolu...,15.0
119,10.1109/ICFHR.2016.0113,ICFHR2016 Competition on the Classification of...,F. Cloppet; V. Églin; V. C. Kieu; D. Stutzmann...,2016,This paper presents the results of the ICFHR20...,26.0


In [6]:
df_ieee = df_ieee.dropna(subset=['DOI'])
df_ieee = df_ieee.dropna(subset=['Citations'])
df_ieee

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/ICTACS62700.2024.10840995,Comparative Analysis of Different Classifiers ...,N. Vyas; G. K. Sethi; S. Singh,2024,Image fusion could adopt the optical and micro...,4.0
2,10.1109/IGARSS47720.2021.9553571,Agricultural SandboxNL: A Crop Parcel Level Da...,V. Kumar; M. Huber; M. Shorachi; B. Rommen; S....,2021,The systematic high temporal coverage of Senti...,1.0
3,10.1109/SEEDA-CECNSM57760.2022.9932913,Open weather data evaluation for crop irrigati...,T. Karamitsou; D. Seventekidis; C. Karapiperis...,2022,Treated wastewater reuse is increasingly impor...,3.0
4,10.1109/JSTARS.2015.2485662,A New European Settlement Map From Optical Rem...,A. J. Florczyk; S. Ferri; V. Syrris; T. Kemper...,2016,An application of a general methodology for pr...,43.0
6,10.1109/IGARSS47720.2021.9553603,"Sen4AgriNet: A Harmonized Multi-Country, Multi...",D. Sykas; I. Papoutsis; D. Zografakis,2021,This work introduces a one of its kind labeled...,8.0
...,...,...,...,...,...,...
116,10.1109/IGARSS.2019.8898926,Remote Sensing For Assessing Drought Insurance...,K. Heidler; A. Fietzke,2019,"In this study, the viability of assessing drou...",1.0
117,10.1109/ACCESS.2019.2933060,"Automated Individual Pig Localisation, Trackin...",J. Cowton; I. Kyriazakis; J. Bacardit,2019,Individual pig tracking is key to stepping awa...,91.0
118,10.1109/JSTARS.2015.2424292,An Approximate Spectral Clustering Ensemble fo...,K. Taşdemir; Y. Moazzen; I. Yildirim,2015,Unsupervised clustering of high spatial resolu...,15.0
119,10.1109/ICFHR.2016.0113,ICFHR2016 Competition on the Classification of...,F. Cloppet; V. Églin; V. C. Kieu; D. Stutzmann...,2016,This paper presents the results of the ICFHR20...,26.0


# Scopus

In [7]:
df_scopus = pd.read_csv(scopus_file)
df_scopus

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISSN,ISBN,CODEN,PubMed ID,Language of Original Document,Document Type,Publication Stage,Open Access,Source,EID
0,"M.K., Pichugin, Mikhail K.; I.A., Gurvich, Iri...","Pichugin, Mikhail K. (36890184300); Gurvich, I...",36890184300; 23012365000; 57195990692; 7005627...,Development of a Diagnostic Algorithm for Dete...,2024,Climate,12,12,224,NaN,...,22251154,NaN,NaN,NaN,English,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85213458202
1,"C.V., Patriche, Cristian Valeriu; B., Roşca, B...","Patriche, Cristian Valeriu (34771966300); Roşc...",34771966300; 57191571017; 55602584700; 5580336...,Digital Mapping of Land Suitability for Main A...,2024,Agronomy,14,12,2828,NaN,...,20734395,NaN,NaN,NaN,English,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85213280216
2,"A.V., Tache, Antonio Valentin; O.C., Popescu, ...","Tache, Antonio Valentin (44661988300); Popescu...",44661988300; 36938096700; 6507767403,Planning Blue–Green Infrastructure for Facing ...,2024,Urban Science,8,4,250,NaN,...,24138851,NaN,NaN,NaN,English,Article,Final,All Open Access; Gold Open Access; Green Accep...,Scopus,2-s2.0-85213219715
3,"D.R.A., Leite, Danilo Rangel Arruda; I., Teixe...","Leite, Danilo Rangel Arruda (57413066600); Tei...",57413066600; 58295142000; 7005147700; 55413934...,Comparative Analysis of CNNs and Vision Transf...,2024,Remote Sensing,16,23,4581,NaN,...,20724292,9780891814184; 0891814183; 9780123694072,NaN,NaN,English,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85211823937
4,"B., Lalić, Branislava; A., Stapleton, Adam; T....","Lalić, Branislava (6602770910); Stapleton, Ada...",6602770910; 57701410900; 36574215100; 55437527...,A comparative analysis of machine learning app...,2024,Environmental Earth Sciences,83,24,679,NaN,...,18666299; 18666280,9783540959908,NaN,NaN,English,Article,Final,NaN,Scopus,2-s2.0-85211208136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,"A., Übelhör, Annegret; S., Munz, Sebastian; S....","Übelhör, Annegret (56442606500); Munz, Sebasti...",56442606500; 36178234200; 6507422325; 6507281785,Evaluation of the CROPGRO model for white cabb...,2015,Scientia Horticulturae,182,NaN,NaN,110.0,...,03044238,NaN,SHRTA,NaN,English,Article,Final,NaN,Scopus,2-s2.0-84949154519
214,"T., Kastner, Thomas; K.H., Erb, Karl Heinz; H....","Kastner, Thomas (35172987400); Erb, Karl Heinz...",35172987400; 7007026767; 7004619776,Global human appropriation of net primary prod...,2015,Journal of Industrial Ecology,19,5,NaN,825.0,...,15309290; 10881980,NaN,JINEF,NaN,English,Article,Final,All Open Access; Green Final Open Access; Gree...,Scopus,2-s2.0-84949531309
215,"T., Kikas, Tambet; R.G.H., Bunce, Robert Geral...","Kikas, Tambet (6602645343); Bunce, Robert Gera...",6602645343; 7006526765; 15042557600; 6603222112,A review of the application of the high nature...,2015,International Journal of Agricultural Resource...,11,2,NaN,143.0,...,14624605,NaN,IJARD,NaN,English,Article,Final,NaN,Scopus,2-s2.0-84946916212
216,"K., Stahl, Kerstin; I., Kohn, Irene; L., De St...","Stahl, Kerstin (14054890700); Kohn, Irene (560...",14054890700; 56005816000; 35291717900; 6701923...,An impact perspective on pan-European drought ...,2015,NaN,NaN,NaN,NaN,329.0,...,NaN,9781138027794,NaN,NaN,English,Conference paper,Final,NaN,Scopus,2-s2.0-84933587957


In [8]:
df_scopus = df_scopus[['DOI', 'Title', 'Authors', 'Year', 'Abstract', 'Cited by']]

df_scopus = df_scopus.rename(columns={
    'Cited by': 'Citations'
})

df_scopus

,DOI,Title,Authors,Year,Abstract,Citations
0,10.3390/cli12120224,Development of a Diagnostic Algorithm for Dete...,"M.K., Pichugin, Mikhail K.; I.A., Gurvich, Iri...",2024,Freezing precipitation and the resultant ice g...,0
1,10.3390/agronomy14122828,Digital Mapping of Land Suitability for Main A...,"C.V., Patriche, Cristian Valeriu; B., Roşca, B...",2024,The scientific evaluation of land potential fo...,0
2,10.3390/urbansci8040250,Planning Blue–Green Infrastructure for Facing ...,"A.V., Tache, Antonio Valentin; O.C., Popescu, ...",2024,Planning for a green–blue infrastructure syste...,0
3,10.3390/rs16234581,Comparative Analysis of CNNs and Vision Transf...,"D.R.A., Leite, Danilo Rangel Arruda; I., Teixe...",2024,The Douro Demarcated Region is fundamental to ...,2
4,10.1007/s12665-024-11982-8,A comparative analysis of machine learning app...,"B., Lalić, Branislava; A., Stapleton, Adam; T....",2024,Observational data of the Earth’s weather and ...,2
...,...,...,...,...,...,...
213,10.1016/j.scienta.2014.11.019,Evaluation of the CROPGRO model for white cabb...,"A., Übelhör, Annegret; S., Munz, Sebastian; S....",2015,"The CROPGRO cabbage model, included in the Dec...",10
214,10.1111/jiec.12238,Global human appropriation of net primary prod...,"T., Kastner, Thomas; K.H., Erb, Karl Heinz; H....",2015,The ongoing globalization process strengthens ...,47
215,10.1504/IJARGE.2015.072902,A review of the application of the high nature...,"T., Kikas, Tambet; R.G.H., Bunce, Robert Geral...",2015,"Since the 1990s, the concept of High Nature Va...",1
216,10.1201/b18077-56,An impact perspective on pan-European drought ...,"K., Stahl, Kerstin; I., Kohn, Irene; L., De St...",2015,"In the past decades, Europe experienced severa...",6


In [9]:
df_scopus = df_scopus.dropna(subset=['DOI'])
df_scopus = df_scopus.loc[df_scopus['Citations'] > 0]
df_scopus

,DOI,Title,Authors,Year,Abstract,Citations
3,10.3390/rs16234581,Comparative Analysis of CNNs and Vision Transf...,"D.R.A., Leite, Danilo Rangel Arruda; I., Teixe...",2024,The Douro Demarcated Region is fundamental to ...,2
4,10.1007/s12665-024-11982-8,A comparative analysis of machine learning app...,"B., Lalić, Branislava; A., Stapleton, Adam; T....",2024,Observational data of the Earth’s weather and ...,2
5,10.1016/j.landusepol.2024.107371,Towards a comprehensive analysis of agricultur...,"E.K., Burchfield, Emily K.; M., Ferro, Marco; ...",2024,Understanding agricultural land systems is par...,1
6,10.1016/j.agee.2024.109143,Earth Observation based multi-scale analysis o...,"M., Machefer, Mélissande; M., Zampieri, Matteo...",2024,Characterizing and quantifying crop diversity ...,7
7,10.21163/GT_2024.192.04,ASSESSING AGRICULTURAL BURNED AREAS USING dNBR...,"R., Samphutthanont, Ratchaphon",2024,This study conducted an assessment of agricult...,1
...,...,...,...,...,...,...
213,10.1016/j.scienta.2014.11.019,Evaluation of the CROPGRO model for white cabb...,"A., Übelhör, Annegret; S., Munz, Sebastian; S....",2015,"The CROPGRO cabbage model, included in the Dec...",10
214,10.1111/jiec.12238,Global human appropriation of net primary prod...,"T., Kastner, Thomas; K.H., Erb, Karl Heinz; H....",2015,The ongoing globalization process strengthens ...,47
215,10.1504/IJARGE.2015.072902,A review of the application of the high nature...,"T., Kikas, Tambet; R.G.H., Bunce, Robert Geral...",2015,"Since the 1990s, the concept of High Nature Va...",1
216,10.1201/b18077-56,An impact perspective on pan-European drought ...,"K., Stahl, Kerstin; I., Kohn, Irene; L., De St...",2015,"In the past decades, Europe experienced severa...",6


# Web of Science

In [10]:
df_wos = pd.read_excel(wos_file)
df_wos

,Publication Type,Authors,Book Authors,Group Authors,Book Group Authors,Researcher Ids,ORCIDs,Book Editors,Author - Arabic,Grant Principal Investigator,...,Copyright,Degree Name,Institution Address,Institution,Dissertation and Thesis Subjects,Author Keywords,Indexed Date,UT (Unique ID),Pubmed Id,Unnamed: 78
0,J,"d'Andrimont, Raphael; Claverie, Martin; Kempen...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-10,WOS:000917162300001,NaN,NaN
1,J,"Arend, Daniel; Junker, Astrid; Scholz, Uwe; Sc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-05-18,WOS:000374649900001,27087305.0,NaN
2,J,"Quaresma, Andreia; Ankenbrand, Markus J.; Garc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-06,WOS:001151164200003,38272945.0,NaN
3,J,"Hughes, Gregory O.; Eatherall, Andrew; Bird, M...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-15,WOS:001135267900001,38054369.0,NaN
4,J,"Champagne, Catherine; White, Jenelle; Berg, Aa...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-22,WOS:000460766100002,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,J,"Derouiche, Emna; Toujani, Rachid; Bdioui, Maro...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-27,ZOOREC:ZOOR16008060965,NaN,NaN
312,J,"Flem, Belinda; Reimann, Clemens; Fabian, Karl;...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-26,WOS:000424846900002,NaN,NaN
313,J,"Martinez-Castaneda, Monica; Feijoo, Claudio",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-11,WOS:001013386300001,NaN,NaN
314,J,"Tang, Y. Sim; Flechard, Chris R.; Daemmgen, Ul...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-05,WOS:000612659000002,NaN,NaN


In [11]:
df_wos = df_wos[['DOI', 'Article Title', 'Authors', 'Publication Year', 'Abstract', 'Times Cited, All Databases']]

df_wos = df_wos.rename(columns={
    'Article Title': 'Title',
    'Publication Year': 'Year',
    'Times Cited, All Databases': 'Citations'
})

df_wos

,DOI,Title,Authors,Year,Abstract,Citations
0,10.5194/essd-15-317-2023,AI4Boundaries: an open AI-ready dataset to map...,"d'Andrimont, Raphael; Claverie, Martin; Kempen...",2023,Field boundaries are at the core of many agric...,10
1,10.1093/database/baw033,PGP repository: a plant phenomics and genomics...,"Arend, Daniel; Junker, Astrid; Scholz, Uwe; Sc...",2016,Plant genomics and phenomics represents the mo...,81
2,10.1038/s41597-024-02962-5,Semi-automated sequence curation for reliable ...,"Quaresma, Andreia; Ankenbrand, Markus J.; Garc...",2024,One of the most critical steps for accurate ta...,9
3,10.1002/ieam.4870,CropLife Europe Crop Development Database: An ...,"Hughes, Gregory O.; Eatherall, Andrew; Bird, M...",2024,There is a regulatory need for crop developmen...,1
4,10.3390/rs11040372,Impact of Soil Moisture Data Characteristics o...,"Champagne, Catherine; White, Jenelle; Berg, Aa...",2019,Soil moisture is often considered a direct way...,28
...,...,...,...,...,...,...
311,NaN,EUROPEAN EEL FISHING EFFORT IN THE MEDITERRANEAN.,"Derouiche, Emna; Toujani, Rachid; Bdioui, Maro...",2023,Fishing effort is one of the most important pa...,0
312,10.1016/j.apgeochem.2017.09.006,Graphical statistics to explore the natural an...,"Flem, Belinda; Reimann, Clemens; Fabian, Karl;...",2018,Plots of cumulative distribution functions (CD...,24
313,10.1016/j.telpol.2023.102574,Use of blockchain in the agri-food value chain...,"Martinez-Castaneda, Monica; Feijoo, Claudio",2023,The Common Agricultural Policy (CAP) of the Eu...,18
314,10.5194/acp-21-875-2021,Pan-European rural monitoring network shows do...,"Tang, Y. Sim; Flechard, Chris R.; Daemmgen, Ul...",2021,A comprehensive European dataset on monthly at...,28


In [12]:
df_wos = df_wos.dropna(subset=['DOI', 'Abstract'])
df_wos = df_wos.loc[df_wos['Year'] != 2025]
df_wos = df_wos.loc[df_wos['Citations'] > 0]
df_wos

,DOI,Title,Authors,Year,Abstract,Citations
0,10.5194/essd-15-317-2023,AI4Boundaries: an open AI-ready dataset to map...,"d'Andrimont, Raphael; Claverie, Martin; Kempen...",2023,Field boundaries are at the core of many agric...,10
1,10.1093/database/baw033,PGP repository: a plant phenomics and genomics...,"Arend, Daniel; Junker, Astrid; Scholz, Uwe; Sc...",2016,Plant genomics and phenomics represents the mo...,81
2,10.1038/s41597-024-02962-5,Semi-automated sequence curation for reliable ...,"Quaresma, Andreia; Ankenbrand, Markus J.; Garc...",2024,One of the most critical steps for accurate ta...,9
3,10.1002/ieam.4870,CropLife Europe Crop Development Database: An ...,"Hughes, Gregory O.; Eatherall, Andrew; Bird, M...",2024,There is a regulatory need for crop developmen...,1
4,10.3390/rs11040372,Impact of Soil Moisture Data Characteristics o...,"Champagne, Catherine; White, Jenelle; Berg, Aa...",2019,Soil moisture is often considered a direct way...,28
...,...,...,...,...,...,...
310,10.1007/s00216-019-01692-7,Performance assessment of digital PCR for the ...,"Cottenet, Geoffrey; Blancpain, Carine; Chuah, ...",2019,Accurate quantitative methods are needed to de...,39
312,10.1016/j.apgeochem.2017.09.006,Graphical statistics to explore the natural an...,"Flem, Belinda; Reimann, Clemens; Fabian, Karl;...",2018,Plots of cumulative distribution functions (CD...,24
313,10.1016/j.telpol.2023.102574,Use of blockchain in the agri-food value chain...,"Martinez-Castaneda, Monica; Feijoo, Claudio",2023,The Common Agricultural Policy (CAP) of the Eu...,18
314,10.5194/acp-21-875-2021,Pan-European rural monitoring network shows do...,"Tang, Y. Sim; Flechard, Chris R.; Daemmgen, Ul...",2021,A comprehensive European dataset on monthly at...,28


# Final Dataset

In [13]:
df_all = pd.concat([df_ieee, df_scopus, df_wos], ignore_index=True)
df_all

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/ICTACS62700.2024.10840995,Comparative Analysis of Different Classifiers ...,N. Vyas; G. K. Sethi; S. Singh,2024,Image fusion could adopt the optical and micro...,4.0
1,10.1109/IGARSS47720.2021.9553571,Agricultural SandboxNL: A Crop Parcel Level Da...,V. Kumar; M. Huber; M. Shorachi; B. Rommen; S....,2021,The systematic high temporal coverage of Senti...,1.0
2,10.1109/SEEDA-CECNSM57760.2022.9932913,Open weather data evaluation for crop irrigati...,T. Karamitsou; D. Seventekidis; C. Karapiperis...,2022,Treated wastewater reuse is increasingly impor...,3.0
3,10.1109/JSTARS.2015.2485662,A New European Settlement Map From Optical Rem...,A. J. Florczyk; S. Ferri; V. Syrris; T. Kemper...,2016,An application of a general methodology for pr...,43.0
4,10.1109/IGARSS47720.2021.9553603,"Sen4AgriNet: A Harmonized Multi-Country, Multi...",D. Sykas; I. Papoutsis; D. Zografakis,2021,This work introduces a one of its kind labeled...,8.0
...,...,...,...,...,...,...
577,10.1007/s00216-019-01692-7,Performance assessment of digital PCR for the ...,"Cottenet, Geoffrey; Blancpain, Carine; Chuah, ...",2019,Accurate quantitative methods are needed to de...,39.0
578,10.1016/j.apgeochem.2017.09.006,Graphical statistics to explore the natural an...,"Flem, Belinda; Reimann, Clemens; Fabian, Karl;...",2018,Plots of cumulative distribution functions (CD...,24.0
579,10.1016/j.telpol.2023.102574,Use of blockchain in the agri-food value chain...,"Martinez-Castaneda, Monica; Feijoo, Claudio",2023,The Common Agricultural Policy (CAP) of the Eu...,18.0
580,10.5194/acp-21-875-2021,Pan-European rural monitoring network shows do...,"Tang, Y. Sim; Flechard, Chris R.; Daemmgen, Ul...",2021,A comprehensive European dataset on monthly at...,28.0


In [14]:
df_all = df_all.drop_duplicates(subset=['DOI'])
df_all = df_all.drop(columns=['Citations'])
df_all = df_all.reset_index(drop=True)
df_all

,DOI,Title,Authors,Year,Abstract
0,10.1109/ICTACS62700.2024.10840995,Comparative Analysis of Different Classifiers ...,N. Vyas; G. K. Sethi; S. Singh,2024,Image fusion could adopt the optical and micro...
1,10.1109/IGARSS47720.2021.9553571,Agricultural SandboxNL: A Crop Parcel Level Da...,V. Kumar; M. Huber; M. Shorachi; B. Rommen; S....,2021,The systematic high temporal coverage of Senti...
2,10.1109/SEEDA-CECNSM57760.2022.9932913,Open weather data evaluation for crop irrigati...,T. Karamitsou; D. Seventekidis; C. Karapiperis...,2022,Treated wastewater reuse is increasingly impor...
3,10.1109/JSTARS.2015.2485662,A New European Settlement Map From Optical Rem...,A. J. Florczyk; S. Ferri; V. Syrris; T. Kemper...,2016,An application of a general methodology for pr...
4,10.1109/IGARSS47720.2021.9553603,"Sen4AgriNet: A Harmonized Multi-Country, Multi...",D. Sykas; I. Papoutsis; D. Zografakis,2021,This work introduces a one of its kind labeled...
...,...,...,...,...,...
461,10.1080/00063657.2019.1618242,Using the first European Breeding Bird Atlas f...,"Herrando, Sergi; Keller, Verena; Bauer, Hans-G...",2019,Capsule The first European Bird Census Council...
462,10.1007/s00216-019-01692-7,Performance assessment of digital PCR for the ...,"Cottenet, Geoffrey; Blancpain, Carine; Chuah, ...",2019,Accurate quantitative methods are needed to de...
463,10.1016/j.apgeochem.2017.09.006,Graphical statistics to explore the natural an...,"Flem, Belinda; Reimann, Clemens; Fabian, Karl;...",2018,Plots of cumulative distribution functions (CD...
464,10.1016/j.telpol.2023.102574,Use of blockchain in the agri-food value chain...,"Martinez-Castaneda, Monica; Feijoo, Claudio",2023,The Common Agricultural Policy (CAP) of the Eu...


# Abstract Evaluation with Gemini

In [16]:
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY2")

client = genai.Client(api_key=API_KEY)
MODEL = "gemini-2.5-pro"

response = client.models.generate_content(
    model=MODEL,
    contents="Explain how AI works in a few words"
)
print(response.text)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


**Finding patterns in data to make predictions.**


In [18]:
class BatchReviewItem(BaseModel):
    id: int
    decision: Annotated[Literal["yes", "no", "maybe"], Field(
        description="Decision about including the paper:"
                    "- 'yes': There is clear evidence that the paper uses, shares, or discusses datasets, and it fully meets ALL inclusion criteria while NOT violating ANY exclusion criteria."
                    "- 'no': There is clear evidence that the paper should be excluded because it fails to meet all inclusion criteria or it meets at least one exclusion criterion."
                    "- 'maybe': The information is insufficient or ambiguous; checking the full text is recommended."
                    "If unsure, or it is needed to check other infos (like the full text) prefer 'maybe' over 'no'.")]
    reason: Annotated[str, Field(description="Short explanation for the decision")]
    data_source: Annotated[str, Field(
        description="Official and recognized name of the data source used in the paper if the dataset is explicitly mentioned."
                    "The dataset must be a real, identifiable resource (dataset, database, or data collection); check online if necessary."
                    "If only a vague or generic description is provided (e.g., 'agricultural data' or 'crop dataset' or 'generated data'), leave it empty."
                    "Use only the proper name of the resource, without adding descriptors about the type of data source (e.g., use 'PlantVillage' instead of 'PlantVillage dataset')."
                    "If multiple data sources are mentioned, insert all of them in the same string separated by ';' (semicolon)."
                    "If the decision is 'no', always leave it empty.")]


class BatchReview(BaseModel):
    items: list[BatchReviewItem]

fallback = BatchReviewItem(id=-1, decision="maybe", reason="No response", data_source="")


def is_open_data_relevant_batch(df_chunk: pd.DataFrame, client, model=None, max_retries=3):
    if model is None:
        model = MODEL if MODEL else "gemini-2.5-flash-lite"

    # Prepare record strings for prompt
    records = []
    for idx, row in df_chunk.iterrows():
        title = "" if pd.isna(row.get("Title")) else str(row.get("Title"))
        abstract = "" if pd.isna(row.get("Abstract")) else str(row.get("Abstract"))
        records.append(f"- id: {int(idx)}\n  Title: {title}\n  Abstract: {abstract}")

    # Build prompt with inclusion/exclusion criteria and instructions
    prompt = (
            "Evaluate each record based on the following Inclusion and Exclusion Criteria and return ONLY JSON that matches the provided schema.\n\n"
            "Inclusion Criteria:\n"
            "(a) The abstract mentions the use or production of datasets, databases, or other data sources.\n"
            "(b) The data sources are related to agriculture or closely related fields (e.g.: climate, soil...).\n\n"
            "Exclusion Criteria:\n"
            "(a) The abstract states that the data are not available or must be requested from the authors.\n"
            "(b) The data used or generated are not free (cost is required).\n"
            "(c) The data are local/national/global (not only european coverage).\n\n"
            "ALL the requirements should be met.\n\n"
            "Output Instructions:\n"
            "- For each provided record, return an object with: id, decision ('yes'|'no'|'maybe'), reason, data_source.\n"
            "- If title or abstract are missing or empty use decision='no' and reason='Title or abstract is missing', data_source=''.\n\n"
            "Records to evaluate:\n"
            + "\n".join(records)
    )

    global _sent_calls_count
    global _completed_evaluations_count
    backoff = 10

    # Error management with exponential backoff
    for _ in range(max_retries):
        _rate_limit_block_until_allowed()
        try:
            _sent_calls_count += 1
            resp = client.models.generate_content(
                model=model,
                contents=prompt,
                config={
                    "response_mime_type": "application/json",
                    "response_schema": BatchReview,
                    "temperature": 0.0
                },
            )
            # Update total evaluations count
            try:
                _completed_evaluations_count += len(df_chunk)
            except Exception as e:
                print(f"[attempt {_sent_calls_count}] Gemini error: {e}")
                _completed_evaluations_count = len(df_chunk)
            print(f"[GEMINI] Completed {_completed_evaluations_count} total evaluations.")
            return resp.parsed.items
        except Exception as e:
            print(f"[attempt {_sent_calls_count}] Gemini error: {e}")
            time.sleep(backoff)
            backoff *= 2
    return []


def _rate_limit_block_until_allowed():
    # Block until the number of calls in the window is below max
    while True:
        now = time.monotonic()
        while _rate_calls_log and (now - _rate_calls_log[0]) >= _RATE_WINDOW_SEC:
            _rate_calls_log.popleft()
        if len(_rate_calls_log) < _RATE_MAX_CALLS:
            _rate_calls_log.append(now)
            return


# Rate limiter: max 2 requests every 60 seconds
_RATE_WINDOW_SEC = 60.0
_RATE_MAX_CALLS = 2
_rate_calls_log = deque()

batch_size = 20  # number of papers per batch
df_evaluated = df_all.copy()

_sent_calls_count = 0
_completed_evaluations_count = 0

# Process dataframe in batches
for start in range(0, len(df_evaluated), batch_size):
    chunk = df_evaluated.iloc[start:start + batch_size]
    items = is_open_data_relevant_batch(chunk, client, model=MODEL)
    mapping = {int(it.id): it for it in items} if items else {}

    decisions = []
    reasons = []
    sources = []
    for idx in chunk.index:
        it = mapping.get(int(idx))
        if it is None:
            decisions.append(fallback.decision)
            reasons.append(fallback.reason)
            sources.append(fallback.data_source)
        else:
            decisions.append(it.decision)
            reasons.append(it.reason)
            sources.append(it.data_source)

    # Update dataframe with results
    df_evaluated.loc[chunk.index, 'Include'] = decisions
    df_evaluated.loc[chunk.index, 'Reason'] = reasons
    df_evaluated.loc[chunk.index, 'Data Source'] = sources

df_evaluated[['DOI', 'Include', 'Reason', 'Data Source']]

[GEMINI] Completed 30 total evaluations.
[GEMINI] Completed 60 total evaluations.
[GEMINI] Completed 90 total evaluations.
[GEMINI] Completed 120 total evaluations.
[GEMINI] Completed 150 total evaluations.
[GEMINI] Completed 180 total evaluations.
[GEMINI] Completed 210 total evaluations.
[GEMINI] Completed 240 total evaluations.
[GEMINI] Completed 270 total evaluations.
[GEMINI] Completed 300 total evaluations.
[GEMINI] Completed 330 total evaluations.
[GEMINI] Completed 360 total evaluations.
[GEMINI] Completed 390 total evaluations.
[GEMINI] Completed 420 total evaluations.
[GEMINI] Completed 450 total evaluations.
[GEMINI] Completed 480 total evaluations.
[GEMINI] Completed 510 total evaluations.
[GEMINI] Completed 540 total evaluations.
[GEMINI] Completed 570 total evaluations.
[attempt 20] Gemini error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head 

,DOI,Include,Reason,Data Source
0,10.1109/JSTARS.2015.2417832,no,The study focuses on using Canadian national s...,
1,10.1109/IGARSS.2018.8518014,yes,The paper mentions using several global datase...,SMAP; NL-DAS; GLDAS; Global Data Assimilation ...
2,10.1109/TEMSMET51618.2020.9557403,no,This is a survey paper that refers to generic ...,
3,10.1109/IDAACS.2017.8095109,yes,The paper uses and discusses multiple global d...,ESA CCI-LC; JRC Land Productivity Dynamics (LP...
4,10.1109/MetroAgriFor52389.2021.9628717,no,"The primary dataset used, Open Plant Phenotypi...",
...,...,...,...,...
891,10.1016/j.dib.2024.110225,maybe,No response,
892,10.3389/fsufs.2023.1216205,maybe,No response,
894,10.1016/j.canrad.2023.06.016,maybe,No response,
897,10.1080/02626667.2021.1890749,maybe,No response,


In [ ]:
df_evaluated.to_csv("data/europe/df_evaluated_europe.csv", index=False)

In [19]:
n_unique = df_evaluated['Data Source'].nunique()
n_unique

139

In [20]:
unique_sources = df_evaluated['Data Source'].unique().tolist()
unique_sources

['',
 'SMAP; NL-DAS; GLDAS; Global Data Assimilation System (GDAS); USGS land-cover',
 'ESA CCI-LC; JRC Land Productivity Dynamics (LPD); Landsat',
 'MODIS global land cover products; 30-m Global Land Cover Dataset',
 'MODIS; AIRS',
 'Advanced Microwave Scanning Radiometer—Earth Observing System; Global Land Data Assimilation System-Noah; ERA-Interim/Land; International Soil Moisture Network; ESA Climate Change Initiative—Soil Moisture (ESA CCI SM)',
 'ALOS-PALSAR; GlobeLand30',
 'MODIS',
 'Global Land Cover Data set; Landsat 5',
 'Tropical Ecology Assessment and Monitoring (TEAM) Network; Landsat; Gridded Population of the World (GPW); Global Roads Open Access Dataset (gROADS)',
 'Sentinel-2',
 'LandSat dataset; USGS Global SRTM dataset; global VCI dataset',
 'WorldClim',
 'GSMaP',
 'EPPO',
 'Sentinel',
 'LandCoverNet',
 'Global Satellite Mapping of Precipitation (GSMaP)',
 'Landsat; MODIS',
 'Black Marble',
 'Penman-Monteith-Leuning-version-2 (PMLv2); Global-dataset-of-solar-induced-

In [23]:
df_final = df_evaluated[df_evaluated["Data Source"] == ""]
df_final = df_final[df_final["Include"] != "no"]
df_final

,DOI,Title,Authors,Year,Abstract,Include,Reason,Data Source
30,10.1109/ICIRCA57980.2023.10220825,Sustainable Fertilizers in Coffee Plantation: ...,R. S; S. R,2023,"To improve soil fertility and crop yields, ear...",maybe,The abstract mentions the use of 'real-world d...,
33,10.1109/PICET60765.2024.10716047,Transforming Agriculture with Machine Learning...,D. P. Akurati; P. Devarahatti; S. K. Konakandl...,2024,Machine learning is a powerful tool for extrac...,maybe,The abstract mentions using 'actual datasets' ...,
34,10.1109/MetroAgriFor52389.2021.9628547,Live weight prediction of cattle using deep im...,A. Ruchay; K. Dorofeev; V. Kalschikov; V. Kolp...,2021,The traditional linear regression algorithm is...,maybe,The abstract mentions using 'real-world datase...,
36,10.1109/TAFE.2024.3447792,Real-Time Plant Disease Identification: Fusion...,P. S. Thakur; S. Chaturvedi; P. Khanna; T. She...,2024,"Climate change, adverse weather conditions, an...",maybe,The abstract mentions the use of five public d...,
39,10.1109/ICACS.2018.8333275,Evaluation of predictive data mining algorithm...,A. Arooj; M. Riaz; M. N. Akram,2018,Agricultural research has strengthened the opt...,maybe,The abstract mentions a 'dataset of agricultur...,
...,...,...,...,...,...,...,...,...
891,10.1016/j.dib.2024.110225,An agro-physiological dataset on industrial to...,"Carucci, Federica; Bregaglio, Simone; Caldarol...",2024,The availability of field experimental data pl...,maybe,No response,
892,10.3389/fsufs.2023.1216205,Global hotspots of climate change adaptation a...,"Costa Jr, Ciniro; Thornton, Philip; Wollenberg...",2023,As countries prioritize climate action under t...,maybe,No response,
894,10.1016/j.canrad.2023.06.016,Real world data in radiotherapy: A data farmin...,"Guihard, S.; Piot, M.; Issoufaly, I.; Giraud, ...",2023,The aim of the data farming project by the Uni...,maybe,No response,
897,10.1080/02626667.2021.1890749,A methodology to downscale water demand data w...,"Zogheib, Charles; Ochoa-Tocachi, Boris F.; Mou...",2021,Mountainous regions are a hotspot for water sc...,maybe,No response,
